In [ ]:
from utils import *
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
import json
from loaders import *

In [ ]:
res_df, scalars = mpc_base_loader()
res_b_df, scalars_b = baseline_loader()

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]
axis = 1

for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    df1["tag"] = df1["ratio"].apply(lambda r: f"K = {r}")
    df1 = df1.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["score"],
            sizemode="sd",
            boxmean=True,
            name="CEM",
            offsetgroup="0",
            legendgroup="0",
            showlegend=(col == 1),
            marker=dict(color=colors[0]),
        ),
        row=1,
        col=col,
    )

    df2 = res_b_df[res_b_df["env"] == env].copy()
    df2 = df2[df2["ratio"].isin(df1["ratio"].unique())]
    df2["tag"] = df2["ratio"].apply(lambda r: f"K = {r}")
    df2 = df2.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df2["tag"],
            y=df2["score"],
            sizemode="sd",
            boxmean=True,
            name="Baseline",
            offsetgroup="1",
            legendgroup="1",
            showlegend=(col == 1),
            marker=dict(color=colors[1]),
        ),
        row=1,
        col=col,
    )

    fig.update_layout(
        **{f"xaxis{axis}": dict(title="Configuration")},
        **({f"yaxis{axis}": dict(title="Score")} if col == 1 else {}),
    )
    axis += 1

fig.update_layout(
    boxmode="group",
    width=1000,
    height=400,
)
fig.write_image("../tex/assets/cem_base.pdf")
fig